# VERİ YÜKLEME VE HİZALAMA

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Bidirectional, LSTM
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Concatenate, LSTM, Bidirectional, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense, Dropout, BatchNormalization, Input, Add, Activation, TimeDistributed # Import TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import tensorflow as tf

In [3]:
base_path = '/content/drive//MyDrive/MELD/'


# Bu dosyada, veri ve ilgili meta bilgileri (kelime haritası, kelime indeksi, maksimum cümle uzunluğu ve etiket indeksi) bulunuyor.
data_path = base_path + 'data_emotion.p'
with open(data_path, 'rb') as file:
    data, W, word_idx_map, vocab, max_sentence_length, label_index = pickle.load(file)

#data içindeki tüm girdiler flat_data_emotion listesine düzleştiriliyor.

flat_data_emotion = [entry for entry in data]

# Veriler, eğitim (train), doğrulama (val) ve test (test) kümeleri olarak ayrılıyor ve her bir kümenin boyutları yazdırılıyor.
train_entries = [entry for entry in flat_data_emotion if entry['split'] == 'train']
val_entries = [entry for entry in flat_data_emotion if entry['split'] == 'val']
test_entries = [entry for entry in flat_data_emotion if entry['split'] == 'test']

print(f"Number of train entries: {len(train_entries)}")
print(f"Number of val entries: {len(val_entries)}")
print(f"Number of test entries: {len(test_entries)}")

# Encode labels
#LabelEncoder kullanılarak etiketler sayısal değerlere kodlanıyor.
#Kodlanan etiketler kategorik forma dönüştürülüyor (one-hot encoding).
#Etiketlerin sayısal değerlere nasıl eşlendiği label_mapping değişkeninde tutuluyor ve yazdırılıyor.
le = LabelEncoder()
y_train = le.fit_transform([entry['y'] for entry in train_entries])
y_val = le.transform([entry['y'] for entry in val_entries])
y_test = le.transform([entry['y'] for entry in test_entries])

y_train_categorical = to_categorical(y_train)
y_val_categorical = to_categorical(y_val)
y_test_categorical = to_categorical(y_test)

label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))

with open(base_path + 'text_glove_average_emotion.pkl', 'rb') as file:
    text_train, text_val, text_test = pickle.load(file)

with open(base_path + 'audio_embeddings_feature_selection_emotion.pkl', 'rb') as file:
    audio_train, audio_val, audio_test = pickle.load(file)

# metin ve ses özellikleri hizalama
def align_features(text_features, audio_features, data_entries, labels):
    text_aligned = []
    audio_aligned = []
    aligned_labels = []
    for entry, label in zip(data_entries, labels):
        key = f"{entry['dialog']}_{entry['utterance']}"
        if key in text_features and key in audio_features:
            text_aligned.append(text_features[key])
            audio_aligned.append(audio_features[key])
            aligned_labels.append(label)
    return np.array(text_aligned), np.array(audio_aligned), np.array(aligned_labels)

text_train_aligned, audio_train_aligned, y_train_aligned = align_features(text_train, audio_train, train_entries, y_train)
text_val_aligned, audio_val_aligned, y_val_aligned = align_features(text_val, audio_val, val_entries, y_val)
text_test_aligned, audio_test_aligned, y_test_aligned = align_features(text_test, audio_test, test_entries, y_test)

# Boyutları kontrol edelim
print(f"text_train_aligned shape: {text_train_aligned.shape}")
print(f"audio_train_aligned shape: {audio_train_aligned.shape}")

print(f"text_val_aligned shape: {text_val_aligned.shape}")
print(f"audio_val_aligned shape: {audio_val_aligned.shape}")

print(f"text_test_aligned shape: {text_test_aligned.shape}")
print(f"audio_test_aligned shape: {audio_test_aligned.shape}")

print(f"y_train_aligned shape: {y_train_aligned.shape}")
print(f"y_val_aligned shape: {y_val_aligned.shape}")
print(f"y_test_aligned shape: {y_test_aligned.shape}")

Number of train entries: 9989
Number of val entries: 1109
Number of test entries: 2610
text_train_aligned shape: (9989, 300)
audio_train_aligned shape: (9989, 1611)
text_val_aligned shape: (1109, 300)
audio_val_aligned shape: (1109, 1611)
text_test_aligned shape: (2610, 300)
audio_test_aligned shape: (2610, 1611)
y_train_aligned shape: (9989,)
y_val_aligned shape: (1109,)
y_test_aligned shape: (2610,)


# PENCERELEME

In [4]:
# Pencere boyutu ve kaydırma adımı
window_size = 5
stride = 1

def create_sliding_windows(data, window_size, stride):
    num_windows = (data.shape[0] - window_size) // stride + 1
    windows = []
    for i in range(0, num_windows * stride, stride):
        window = data[i:i+window_size]
        windows.append(window)
    return np.array(windows)

def create_label_windows(labels, window_size, stride):
    num_windows = (len(labels) - window_size) // stride + 1
    windows = []
    for i in range(0, num_windows * stride, stride):
        window = labels[i:i+window_size]
        windows.append(window[-1])  # Sadece son pencereyi kullan
    return np.array(windows)

# Metin ve ses verileri için kaydırma pencereleri oluşturma
text_train_windows = create_sliding_windows(text_train_aligned, window_size, stride)
audio_train_windows = create_sliding_windows(audio_train_aligned, window_size, stride)
text_val_windows = create_sliding_windows(text_val_aligned, window_size, stride)
audio_val_windows = create_sliding_windows(audio_val_aligned, window_size, stride)
text_test_windows = create_sliding_windows(text_test_aligned, window_size, stride)
audio_test_windows = create_sliding_windows(audio_test_aligned, window_size, stride)

# Etiket pencerelerini oluşturma
y_train_windows = create_label_windows(y_train_aligned, window_size, stride)
y_val_windows = create_label_windows(y_val_aligned, window_size, stride)
y_test_windows = create_label_windows(y_test_aligned, window_size, stride)

# Etiketleri kategorik hale getirme
y_train_categorical = to_categorical(y_train_windows)
y_val_categorical = to_categorical(y_val_windows)
y_test_categorical = to_categorical(y_test_windows)

# Boyutları kontrol etme
print(f"text_train_windows shape: {text_train_windows.shape}")
print(f"audio_train_windows shape: {audio_train_windows.shape}")
print(f"text_val_windows shape: {text_val_windows.shape}")
print(f"audio_val_windows shape: {audio_val_windows.shape}")
print(f"text_test_windows shape: {text_test_windows.shape}")
print(f"audio_test_windows shape: {audio_test_windows.shape}")

print(f"y_train_windows shape: {y_train_windows.shape}")
print(f"y_val_windows shape: {y_val_windows.shape}")
print(f"y_test_windows shape: {y_test_windows.shape}")

text_train_windows shape: (9985, 5, 300)
audio_train_windows shape: (9985, 5, 1611)
text_val_windows shape: (1105, 5, 300)
audio_val_windows shape: (1105, 5, 1611)
text_test_windows shape: (2606, 5, 300)
audio_test_windows shape: (2606, 5, 1611)
y_train_windows shape: (9985,)
y_val_windows shape: (1105,)
y_test_windows shape: (2606,)
